In [15]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import rc_context
import bbknn
import re
import json
import os
import rpy2
import anndata
from datetime import date
from scipy.stats import binom_test
from datetime import datetime
import requests

# YYYY-MM-DD
today = date.today()
today = today.strftime("%Y-%m-%d")

import warnings
warnings.filterwarnings('ignore')

# Read in peaks matrix and tidy it

In [2]:
%%time

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
    
print(f'{current_time}:...reading peak file...takes 5 mins...')
peaks=pd.read_csv('/nfs/team205/heart/anndata_objects/Foetal/multiome_ATAC/ArchR/project_output/PeakMatrix/Foetal_celltype-by-Peak.csv')
    
peaks.columns = peaks.columns.str.replace('Unnamed: 0', 'fine_grain')
peaks=peaks.set_index(peaks['fine_grain'])
peaks=peaks.drop(columns=['fine_grain'])

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f'{current_time}:...done.')

21:09:29:...reading peak file...takes 5 mins...
21:14:24:...done.
CPU times: user 4min 46s, sys: 7.1 s, total: 4min 53s
Wall time: 4min 54s


In [3]:
%%time
all_peaks=pd.DataFrame(peaks.columns, columns=['peak'])
all_peaks['chrom']=all_peaks['peak'].str.split(':',expand=True)[0]
all_peaks['peak_window']=all_peaks['peak'].str.split(':',expand=True)[1]
all_peaks['start']=all_peaks['peak_window'].str.split('_',expand=True)[0].astype(int)
all_peaks['end']=all_peaks['peak_window'].str.split('_',expand=True)[1].astype(int)
all_peaks=all_peaks.set_index(['peak'])
all_peaks.head(4)

CPU times: user 5.06 s, sys: 136 ms, total: 5.2 s
Wall time: 5.2 s


,chrom,peak_window,start,end
peak,,,,
chr1:794775_795275,chr1,794775_795275,794775,795275
chr1:817100_817600,chr1,817100_817600,817100,817600
chr1:817796_818296,chr1,817796_818296,817796,818296
chr1:818511_819011,chr1,818511_819011,818511,819011


# Set parameters for subsequent analysis

### Set Cell types

In [25]:
# get list of celltypes for subsequent analyses
cell_types=peaks.index.unique().tolist()

# remove celltypes not for analysis
cell_types.remove('Platelets')
cell_types.remove('VentricularCardiomyocytesCycling')
cell_types.remove('AtrialCardiomyocytesCycling')

#cell_types=cell_types[25:27] # reduced while developing

print(len(cell_types))

44


### Set n_permutations

In [26]:
# Set number of permutations
n_permutations=1000 # reduced while developing
print(n_permutations)

1000


### Set threshold for binarisation

In [27]:
# Set threshold for binarisation
threshold=0.1
threshold_for_filename=str(threshold).replace('.','p')
print(threshold_for_filename)

0p1


# Prepare manually curated CHD gwas SNP data

In [ ]:
# manually collected list of SNPS which as signifcant at the genome wide level for a variety of CHD traits

snps=pd.read_csv('/nfs/team205/heart/CHD_GWAS/chd_gwas_SNPs.csv').sort_values('simplified_classification')
snps.head(3)

In [24]:
%%time

# Use Ensembl APU to add chromosomal position of each of the SNPs

pos=[]
chrom=[]


for snp in range(len(snps['SNP'])):
    server = "https://rest.ensembl.org"
    ext = f"/variation/human/{snps['SNP'][snp]}?"
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    decoded = r.json()
    if len(decoded)>0 and "mappings" in decoded:
        pos.append(decoded['mappings'][0]['start'])
        chrom.append(decoded['mappings'][0]['seq_region_name'])
snps['pos']=pos
snps['chrom']=chrom
snps.set_index('SNP',inplace=True)
snps.to_csv('/nfs/team205/heart/CHD_GWAS/chd_gwas_SNPs_with_pos.csv')

CPU times: user 2.27 s, sys: 44.1 ms, total: 2.32 s
Wall time: 30.8 s


In [31]:
phenotypes=snps.simplified_classification.unique().tolist()
for phenotype in phenotypes:
    tmp=snps[snps['simplified_classification']==phenotype]
    tmp.to_csv(f'/nfs/team205/heart/CHD_GWAS/chd_gwas_SNPs_with_pos-{phenotype}.csv')

In [34]:
snps_path='/nfs/team205/heart/CHD_GWAS/'

files=os.listdir(snps_path)
for phenotype in phenotypes:
    file = [f for f in files if f"{phenotype}" in f]
    file = str(file[0])
    snps_df=pd.read_csv(f'{snps_path}{file}')
snps_df

,SNP,simplified_classification,phenotype,study,pos,chrom
0,rs76774446,vascular,thoracic vessels,Lahm2021,40807221,14
1,rs143638934,vascular,TGA,Lahm2021,40922047,14
2,rs149890280,vascular,TGA,Lahm2021,104604349,5
3,rs150246290,vascular,TGA,Lahm2021,9320073,6
4,rs77094733,vascular,TGA,Lahm2021,73601791,9
5,rs117527287,vascular,thoracic vessels,Lahm2021,51868281,5
6,rs11874,vascular,thoracic vessels,Lahm2021,80475711,X
7,rs17677363,vascular,thoracic vessels,Lahm2021,52539853,10
8,rs148563140,vascular,TGA,Lahm2021,44161476,6
9,rs149467721,vascular,TGA,Lahm2021,115946361,5


In [38]:
%%time
# make a table of metadata about the SNP files
md_simplified_classifcation=[]
md_n_SNPs=[]

snps_path='/nfs/team205/heart/CHD_GWAS/'

files=os.listdir(snps_path)

for phenotype in phenotypes:
    file = [f for f in files if f"{phenotype}" in f]
    file = str(file[0])
    snps_df=pd.read_csv(f'{snps_path}{file}')
    snps_df=snps_df.set_index('SNP')
    md_simplified_classifcation.append(phenotype)
    md_n_SNPs.append(len(snps_df))

md_dict={
    'simplified_classifcation':md_simplified_classifcation,
    'n_SNPs':md_n_SNPs
}

SNP_md=pd.DataFrame(md_dict)

SNP_md=SNP_md.set_index('simplified_classifcation')

n_traits=len(phenotypes)

SNP_md.sort_values('n_SNPs',ascending=False)

CPU times: user 17.4 ms, sys: 31 µs, total: 17.5 ms
Wall time: 23.8 ms


,n_SNPs
simplified_classifcation,
obstructive,21
vascular,10
LH,8
NOS,8
septal,8
RH,1


# Evaluate whether SNPs are found in peaks using the all_peaks file

In [36]:
!mkdir /nfs/team205/heart/CHD_GWAS/SNP_mapped_to_peaks/

In [40]:
%%time
# Add on a column for each trait, indicating whether a SNP from that trait falls within a peak. This file varies with the which traits are assessed
files=os.listdir(snps_path)

if os.path.isfile(f'/nfs/team205/heart/CHD_GWAS/SNP_mapped_to_peaksall_peaks_with_SNPs_for_{n_traits}_traits_incremental.csv')==False:
    print(f'...a SNPs in peaks file for this set of traits is NOT found, making one now...')
    for phenotype in phenotypes:

            file = [f for f in files if f"{phenotype}" in f]
            file = str(file[0])

            snps_df=pd.read_csv(f'{snps_path}{file}')
            snps_df=snps_df.set_index('SNP')

            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            print(f'{current_time}_{phenotype}:...evaluating for SNPs in peaks...')

            all_peaks[phenotype]=0
            for snp in range(len(snps_df["chrom"])): # This loop incrementally adds 1 if there is a SNP within a peak (open or closed)
                all_peaks[phenotype][
                    (all_peaks['chrom']==snps_df["chrom"][snp])
                    &
                    (all_peaks['start'] <= snps_df['pos'][snp])
                    &
                    (all_peaks['end'] >= snps_df['pos'][snp])
                ]+=1 # Adds one for each SNP which falls inside a peak
    all_peaks.to_csv(f'/nfs/team205/heart/CHD_GWAS/SNP_mapped_to_peaksall_peaks_with_SNPs_for_{n_traits}_traits_incremental.csv')
else:
    all_peaks=pd.read_csv(f'/nfs/team205/heart/CHD_GWAS/SNP_mapped_to_peaksall_peaks_with_SNPs_for_{n_traits}_traits_incremental.csv',index_col='peak')
    print(f'...SNPs in peaks file already exists, reading it in...')

all_peaks

...a SNPs in peaks file for this set of traits is NOT found, making one now...
22:01:20_LH:...evaluating for SNPs in peaks...
22:01:27_NOS:...evaluating for SNPs in peaks...
22:01:33_RH:...evaluating for SNPs in peaks...
22:01:33_obstructive:...evaluating for SNPs in peaks...
22:01:50_septal:...evaluating for SNPs in peaks...
22:01:56_vascular:...evaluating for SNPs in peaks...
CPU times: user 37.6 s, sys: 138 ms, total: 37.7 s
Wall time: 38.4 s


,chrom,peak_window,start,end,LH,NOS,RH,obstructive,septal,vascular
peak,,,,,,,,,,
chr1:794775_795275,chr1,794775_795275,794775,795275,0,0,0,0,0,0
chr1:817100_817600,chr1,817100_817600,817100,817600,0,0,0,0,0,0
chr1:817796_818296,chr1,817796_818296,817796,818296,0,0,0,0,0,0
chr1:818511_819011,chr1,818511_819011,818511,819011,0,0,0,0,0,0
chr1:820996_821496,chr1,820996_821496,820996,821496,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
chrX:155768443_155768943,chrX,155768443_155768943,155768443,155768943,0,0,0,0,0,0
chrX:155820075_155820575,chrX,155820075_155820575,155820075,155820575,0,0,0,0,0,0
chrX:155880511_155881011,chrX,155880511_155881011,155880511,155881011,0,0,0,0,0,0


# Make a table of peaks x cell for each cell type, binarise using a threshold, then make 1000 permutations, then overwrite the file

In [41]:
%%time

# makes cell x peak matrix, including 1000 permutations
# NB the SNPs in peaks file needs is specific to the defined set of peaks, which will vary accoring to threshold for binarisation, so this all peaks file needs to be saved according to threshold used

permutations=range(n_permutations)

bin_mat_path='/nfs/team205/heart/EBI_GWAS/binarised_permutation_matrices/'

for cell_type in cell_types:
    if os.path.isfile(f'{bin_mat_path}{cell_type}_{n_permutations}_permutations_bin_threshold_{threshold_for_filename}_matrix.csv') == False:

        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        
        print(f'{current_time}...generating binarised matrix for {cell_type}...')

        cell_df=pd.DataFrame(peaks.loc[cell_type])
        cell_df['peak'] = cell_df.index
        cell_df['chrom']=cell_df['peak'].str.split(':',expand=True)[0]
        cell_df['peak_window']=cell_df['peak'].str.split(':',expand=True)[1]
        cell_df['start']=cell_df['peak_window'].str.split('_',expand=True)[0].astype(int)
        cell_df['end']=cell_df['peak_window'].str.split('_',expand=True)[1].astype(int)
        cell_df[f'{cell_type}_binarised_real']=cell_df[cell_type].ge(threshold).astype(int)

        for permutation in permutations:
            cell_df[f'{cell_type}_binarised_permutation_{permutation}'] = np.random.permutation(cell_df[f'{cell_type}_binarised_real'])
        cell_df=cell_df.filter(regex=cell_type+'_')
        cell_df.to_csv(f'{bin_mat_path}{cell_type}_{n_permutations}_permutations_bin_threshold_{threshold_for_filename}_matrix.csv',index=True)
    else:
        print(f'NOT generating binarised matrix for {cell_type} since it already exists')
print('finished')

NOT generating binarised matrix for VentricularConductionSystemDistal since it already exists
NOT generating binarised matrix for DendriticCells since it already exists
NOT generating binarised matrix for GreatVesselAdventitialFibroblasts since it already exists
NOT generating binarised matrix for GlialCells since it already exists
NOT generating binarised matrix for AtrialCardiomyocytesRight since it already exists
NOT generating binarised matrix for ParasympatheticNeurons since it already exists
NOT generating binarised matrix for CoronaryEndothelialCellsVenous since it already exists
NOT generating binarised matrix for NeuronProgenitors since it already exists
NOT generating binarised matrix for MastCells since it already exists
NOT generating binarised matrix for ProBCells since it already exists
NOT generating binarised matrix for LymphaticEndothelialCells since it already exists
NOT generating binarised matrix for NaturalKillerCells since it already exists
NOT generating binarise

# Read in Binarised Matrix for each cell type, join it to the all_peaks file which has trait info (binding on the index [peaks]), then save these joined binarised matrix files (now with added trait info) in a different directory

In [ ]:
%%time

joined_bin_mat_path='/nfs/team205/heart/CHD_GWAS/binarised_permutation_matrices_joined_incremental/'

for cell_type in cell_types:
    if os.path.isfile(f'{joined_bin_mat_path}{cell_type}_{n_permutations}_permutations_bin_threshold_{threshold_for_filename}_matrix_joined.csv') == False:    
        
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(f'{current_time}: {cell_type}: making bin matrix with trait for this cell')

        # read in binary matrices which have already been made
        binarised_matrix=pd.read_csv(f'{bin_mat_path}{cell_type}_{n_permutations}_permutations_bin_threshold_{threshold_for_filename}_matrix.csv')

        # tidy and add the columns we need
        binarised_matrix.rename(columns = {'Unnamed: 0':'peak'}, inplace = True)
        binarised_matrix=binarised_matrix.set_index(binarised_matrix.iloc[:,0])
        binarised_matrix=binarised_matrix.drop(columns=['peak'])
        
        # add on the columns indicating whether SNPs are in peaks
        binarised_matrix=binarised_matrix.join(all_peaks)

        # save this modified binary matrix to a separate directory
        binarised_matrix.to_csv(f'{joined_bin_mat_path}{cell_type}_{n_permutations}_permutations_bin_threshold_{threshold_for_filename}_matrix_joined.csv')
    else:
        print(f'NOT generating joined binarised matrix for {cell_type} since it already exists')


print('finished')

22:06:29: VentricularConductionSystemDistal: making bin matrix with trait for this cell
22:08:42: DendriticCells: making bin matrix with trait for this cell
22:10:53: GreatVesselAdventitialFibroblasts: making bin matrix with trait for this cell
22:13:03: GlialCells: making bin matrix with trait for this cell
22:15:12: AtrialCardiomyocytesRight: making bin matrix with trait for this cell


# Calculate enrichment of cell types for different traits

In [ ]:
%%time
# Find the proportion of all peaks which are open in this celltype

output_path='/nfs/team205/heart/EBI_GWAS/enrichment_output/'

list_of_output_dfs=[]

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f'...================================================================...') 
print(f'...{current_time}:STARTING')
print(f'...================================================================...') 

# make some empty lists
proportion_of_SNPs_found_in_celltype_specific_open_peaks=[]
proportion_of_all_open_peaks_found_in_this_celltype=[]

n_times_proportions_of_SNPs_in_permuted_open_peaks_greater_than_observed_proportion=[]
mean_proportions_of_SNPs_in_open_peaks=[]
p_values=[]
phenotype_list=[]
cell_type_list=[]
n_SNPs_list=[]

cell_types_done=[]
n_cell_types_total=len(cell_types)

permutations=range(n_permutations)

print('...evaluating '+str(len(phenotypes))+' traits, across '+str(len(cell_types))+' cell types...')

for cell_type in cell_types:
    
    cell_types_done.append(cell_type)
    n_cell_types_done=len(cell_types_done)
    n_cell_types_remaining=n_cell_types_total-n_cell_types_done
    
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print(f'...================================================================...')    
    print(f'...{current_time}: {n_cell_types_remaining+1} of {n_cell_types_total} cell types remaining. Reading binarised matrix for {cell_type}...')
    print(f'...================================================================...') 

    cell_bin_mat=pd.read_csv(f'{joined_bin_mat_path}{cell_type}_{n_permutations}_permutations_bin_threshold_{threshold_for_filename}_matrix_joined.csv',index_col='peak')
    
    prop_bins_in_this_cell_type=(len(cell_bin_mat[cell_bin_mat[f'{cell_type}_binarised_real']==1]))/len(cell_bin_mat)
            
    
    for phenotype in phenotypes:
                
        # grab some metadata
        n_SNPs=SNP_md.loc[phenotype]['n_SNPs']
        
        # add columns which won't change until we run a new cell_type
        proportion_of_all_open_peaks_found_in_this_celltype.append(prop_bins_in_this_cell_type)
        cell_type_list.append(cell_type)

        # add columns which won't change until we run a new phenotype
        n_SNPs_list.append(n_SNPs)
        phenotype_list.append(phenotype)
        
        # subset to just open regions for this cell type
        # find the proportion of SNPs for this trait that lie within this cell types open peaks
        observed_proportion=(cell_bin_mat[phenotype][cell_bin_mat[f'{cell_type}_binarised_real']==1].sum())/(cell_bin_mat[phenotype].sum())

        proportion_of_SNPs_found_in_celltype_specific_open_peaks.append(observed_proportion)
        
        proportion_of_SNPs_found_in_permutations_of_celltype_specific_open_peaks=[]

        for permutation in permutations:
            proportion_of_SNPs_found_in_permutations_of_celltype_specific_open_peaks.append(cell_bin_mat[phenotype][cell_bin_mat[f'{cell_type}_binarised_permutation_{permutation}']==1].sum()/cell_bin_mat[phenotype].sum())

        proportions_of_SNPs_in_permuted_open_peaks_greater_than_observed_proportion = [i for i in proportion_of_SNPs_found_in_permutations_of_celltype_specific_open_peaks if i >= observed_proportion]
        
        n_times_proportions_of_SNPs_in_permuted_open_peaks_greater_than_observed_proportion.append(len(proportions_of_SNPs_in_permuted_open_peaks_greater_than_observed_proportion))

        p_values.append(len(proportions_of_SNPs_in_permuted_open_peaks_greater_than_observed_proportion)/len(permutations)) # p val is simply the proportion of null hypotheses 'observations' greater than the actual observed proportion

        mean_proportions_of_SNPs_in_open_peaks.append(sum(proportion_of_SNPs_found_in_permutations_of_celltype_specific_open_peaks)/len(proportion_of_SNPs_found_in_permutations_of_celltype_specific_open_peaks))
        
        # Plot histograms for each cell type
#        plt.rcParams["figure.figsize"] = (20,10)
#        plt.rcParams["figure.dpi"] = 300

#        plt.hist(proportion_of_SNPs_found_in_permutations_of_celltype_specific_open_peaks,
#                 bins=100,color='red',
#                 range=(0,1),
#                 histtype='stepfilled',edgecolor='none')
#        plt.axvline(x=observed_proportion, color='blue', linestyle='--')
#        plt.legend(['null: proportion of SNPs falling in randomly shuffled OC regions','observed: proportion of SNPs falling cell-type specific OC regions'])
#        plt.title('cell type: '+cell_type+', trait: '+phenotype+', phenotype: '+phenotype+', threshold for binarisation: '+threshold_for_filename)
#        plt.savefig(f'{output_path}{phenotype}_{cell_type}_{threshold_for_filename}_SNP_enrichment.png')
#        plt.clf() #clears the current plobt

        
output_dict={
    'cell_type':cell_type_list,
    'proportion_of_all_open_peaks_found_in_this_celltype':proportion_of_all_open_peaks_found_in_this_celltype,
    'proportion_of_SNPs_found_in_celltype_specific_open_peaks':proportion_of_SNPs_found_in_celltype_specific_open_peaks,
    'mean_proportions_of_SNPs_in_open_peaks':mean_proportions_of_SNPs_in_open_peaks,
    'n_times_proportions_of_SNPs_in_permuted_open_peaks_greater_than_observed_proportion':n_times_proportions_of_SNPs_in_permuted_open_peaks_greater_than_observed_proportion,
    'p_value':p_values,
    'n_SNPs':n_SNPs_list,
    'phenotype':phenotype_list}

output_df=pd.DataFrame(output_dict)

list_of_output_dfs.append(output_df)
combined_output_df=pd.concat(list_of_output_dfs)
combined_output_df=combined_output_df.sort_values(by=['phenotype'])
combined_output_df=combined_output_df.set_index('cell_type')
combined_output_df.to_csv(f'{output_path}{threshold_for_filename}_all_traits_summary.csv')

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f'...================================================================...')    
print(f'...{current_time}:FINSIHED')
print(f'...================================================================...')